<h1 style="text-align:center;">Data Fetch & Add Target</h1>

---

### Import Libraries

In [65]:
# type: ignore
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Prevent line-wrapping
import warnings
warnings.filterwarnings("ignore")
from functools import reduce
import yfinance as yf
from compute_target import compute_tunh

### 1. Fetch Bitcoin

In [86]:
y_symbols = 'BTC-USD'

btc_OHLC_online = yf.Ticker(y_symbols).history('max')
btc_OHLC_online = btc_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
btc_OHLC_online.reset_index(inplace=True)
btc_OHLC_online['Date'] = pd.to_datetime(btc_OHLC_online['Date']).dt.date
print(btc_OHLC_online.head())

         Date        Open        High         Low       Close    Volume
0  2014-09-17  465.864014  468.174011  452.421997  457.334015  21056800
1  2014-09-18  456.859985  456.859985  413.104004  424.440002  34483200
2  2014-09-19  424.102997  427.834991  384.532013  394.795990  37919700
3  2014-09-20  394.673004  423.295990  389.882996  408.903992  36863600
4  2014-09-21  408.084991  412.425995  393.181000  398.821014  26580100


In [87]:
# Import old data from 2010 to 2014-09-16
btc_OHLC_offline = pd.read_csv('data/Bitcoin-2014-09-16_to_Jul-17-2010.csv')
btc_OHLC_offline['Date'] = pd.to_datetime(btc_OHLC_offline['Date']).dt.date
btc_OHLC_offline = btc_OHLC_offline.drop(columns=['Market Cap'], axis=1)
print(btc_OHLC_offline.head())

         Date     Open     High      Low    Close        Volume
0  2014-09-16  475.371  475.642  465.128  465.955  1.700966e+07
1  2014-09-15  477.892  478.620  474.022  475.434  1.383323e+07
2  2014-09-14  479.002  479.853  476.119  477.745  1.357271e+07
3  2014-09-13  477.753  482.117  475.098  478.879  1.556195e+07
4  2014-09-12  479.749  479.749  473.009  477.593  1.611576e+07


In [88]:
# Combining both dataframes
btc_OHLC = pd.concat([btc_OHLC_online, btc_OHLC_offline],join="outer", ignore_index=True)
btc_OHLC.sort_values(by='Date', inplace = True)
btc_OHLC['Date'] = pd.to_datetime(btc_OHLC['Date'])
btc_OHLC

# Define list of DataFrames
dfs = [btc_OHLC]

# Merge DataFrames into one
df_btc = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), dfs)

In [89]:
# Sort by date
df_btc.sort_values(by='Date', inplace = True)
df_btc = compute_tunh(df_btc)
print(df_btc)

           Date          Open          High           Low         Close        Volume  tunh
5380 2010-07-17      0.050000      0.050000      0.050000      0.050000  0.000000e+00   1.0
5379 2010-07-18      0.085800      0.085800      0.085800      0.085800  0.000000e+00  82.0
5378 2010-07-19      0.080800      0.080800      0.080800      0.080800  0.000000e+00  81.0
5377 2010-07-20      0.074700      0.074700      0.074700      0.074700  0.000000e+00   1.0
5376 2010-07-21      0.079200      0.079200      0.079200      0.079200  0.000000e+00  79.0
...         ...           ...           ...           ...           ...           ...   ...
3853 2025-04-05  83844.703125  84207.015625  82377.734375  83504.796875  1.438080e+10   NaN
3854 2025-04-06  83504.507812  83704.718750  77097.742188  78214.484375  3.629485e+10   NaN
3855 2025-04-07  78221.335938  81119.062500  74436.679688  79235.335938  9.126242e+10   NaN
3856 2025-04-08  79218.476562  80823.890625  76198.023438  76271.953125  4.83145

In [90]:
df_btc.to_csv("data/final_data_btc.csv", index=False)

### 2. Fetch PAXG

In [91]:
y_symbols = 'PAXG-USD'

paxg_OHLC_online = yf.Ticker(y_symbols).history('max')
paxg_OHLC_online = paxg_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
paxg_OHLC_online.reset_index(inplace=True)
paxg_OHLC_online['Date'] = pd.to_datetime(paxg_OHLC_online['Date']).dt.date
paxg_OHLC_online.sort_values(by='Date', inplace = True)
paxg_OHLC_online = compute_tunh(paxg_OHLC_online)
print(paxg_OHLC_online)

           Date         Open         High          Low        Close     Volume  tunh
0    2019-09-26  1522.969238  1540.213257  1499.900024  1516.510620    5337186   1.0
1    2019-09-27  1515.437378  1547.361206  1502.289062  1528.739990    2583554   1.0
2    2019-09-28  1525.926392  1553.475098  1508.814087  1525.717896    2243053   8.0
3    2019-09-29  1526.947998  1551.402344  1511.673950  1528.866577    1602218   7.0
4    2019-09-30  1528.866577  1538.616821  1497.364990  1513.878052    3023598   1.0
...         ...          ...          ...          ...          ...        ...   ...
2018 2025-04-05  3053.608154  3068.875977  3047.419922  3067.476074   33405009   1.0
2019 2025-04-06  3067.491943  3069.281738  2991.297119  2999.474121   66856845   3.0
2020 2025-04-07  2999.474121  3063.089844  2990.072266  3005.760498  153272254   2.0
2021 2025-04-08  3005.781250  3034.911377  2993.993164  2998.993652   60268815   1.0
2022 2025-04-09  2998.721680  3085.822021  2994.058105  3079.8447

In [92]:
paxg_OHLC_online.to_csv("data/final_data_paxg.csv", index=False)

### 3. Fetch ETH

In [93]:
y_symbols = 'ETH-USD'

eth_OHLC_online = yf.Ticker(y_symbols).history('max')
eth_OHLC_online = eth_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
eth_OHLC_online.reset_index(inplace=True)
eth_OHLC_online['Date'] = pd.to_datetime(eth_OHLC_online['Date']).dt.date
eth_OHLC_online.sort_values(by='Date', inplace = True)
eth_OHLC_online = compute_tunh(eth_OHLC_online)
print(eth_OHLC_online)

           Date         Open         High          Low        Close       Volume  tunh
0    2017-11-09   308.644989   329.451996   307.056000   320.884003    893249984   5.0
1    2017-11-10   320.670990   324.717987   294.541992   299.252991    885985984   3.0
2    2017-11-11   298.585999   319.453003   298.191986   314.681000    842300992   2.0
3    2017-11-12   314.690002   319.153015   298.513000   307.907990   1613479936   1.0
4    2017-11-13   307.024994   328.415009   307.024994   316.716003   1041889984   1.0
...         ...          ...          ...          ...          ...          ...   ...
2704 2025-04-05  1815.344971  1826.298584  1767.513550  1805.973267   6374712479   NaN
2705 2025-04-06  1805.963013  1815.574829  1539.437988  1576.728027  22154445576   NaN
2706 2025-04-07  1576.949829  1634.041138  1415.373535  1555.240967  46073959047   NaN
2707 2025-04-08  1554.932129  1617.339844  1447.610107  1472.553101  21315312919   NaN
2708 2025-04-09  1471.944214  1489.208496  

In [94]:
# Save to CSV
eth_OHLC_online.to_csv("data/final_data_eth.csv", index=False)

### 4. Fetch BNB

In [95]:
y_symbols = 'BNB-USD'

bnb_OHLC_online = yf.Ticker(y_symbols).history('max')
bnb_OHLC_online = bnb_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
bnb_OHLC_online.reset_index(inplace=True)
bnb_OHLC_online['Date'] = pd.to_datetime(bnb_OHLC_online['Date']).dt.date
bnb_OHLC_online.sort_values(by='Date', inplace = True)
bnb_OHLC_online = compute_tunh(bnb_OHLC_online)
print(bnb_OHLC_online)

           Date        Open        High         Low       Close      Volume  tunh
0    2017-11-09    2.053140    2.174230    1.893940    1.990770    19192200  24.0
1    2017-11-10    2.007730    2.069470    1.644780    1.796840    11155000  21.0
2    2017-11-11    1.786280    1.917750    1.614290    1.670470     8178150  14.0
3    2017-11-12    1.668890    1.672800    1.462560    1.519690    15298700   1.0
4    2017-11-13    1.526010    1.735020    1.517600    1.686620    12238800   1.0
...         ...         ...         ...         ...         ...         ...   ...
2704 2025-04-05  597.579285  598.546326  589.932983  593.114868  1008300251   NaN
2705 2025-04-06  593.138794  593.461487  545.322693  554.594910  1679604717   NaN
2706 2025-04-07  554.594910  564.792419  522.715759  554.483398  2852045942   1.0
2707 2025-04-08  554.485901  564.938232  548.141235  553.467407  1676208165   NaN
2708 2025-04-09  553.348877  558.932190  535.629456  555.699280  1779421440   NaN

[2709 rows x 7 

In [96]:
bnb_OHLC_online.to_csv("data/final_data_bnb.csv", index=False)

### 5. Fetch AAVE

In [97]:
y_symbols = 'AAVE-USD'

aave_OHLC_online = yf.Ticker(y_symbols).history('max')
aave_OHLC_online = aave_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
aave_OHLC_online.reset_index(inplace=True)
aave_OHLC_online['Date'] = pd.to_datetime(aave_OHLC_online['Date']).dt.date
aave_OHLC_online.sort_values(by='Date', inplace = True)
aave_OHLC_online = compute_tunh(aave_OHLC_online)
print(aave_OHLC_online)

           Date        Open        High         Low       Close     Volume  tunh
0    2020-10-02    0.000000    0.516571    0.000000    0.516571          0   1.0
1    2020-10-03    0.523836   65.305939    0.523801   53.151489          0  38.0
2    2020-10-04   53.179859   55.070354   50.688992   52.675034          0   1.0
3    2020-10-05   52.675034   55.112358   49.787899   53.219242          0   8.0
4    2020-10-06   53.291969   53.402271   40.734577   42.401600     583091   6.0
...         ...         ...         ...         ...         ...        ...   ...
1646 2025-04-05  151.833847  154.877975  149.600723  152.586380  126561823   NaN
1647 2025-04-06  152.585938  152.598663  126.753128  129.087051  224729974   NaN
1648 2025-04-07  129.085556  135.252090  115.034019  132.002640  518877924   1.0
1649 2025-04-08  132.002640  135.808868  123.735817  125.094566  325864837   NaN
1650 2025-04-09  125.084282  129.806198  120.369865  126.432327  349685472   NaN

[1651 rows x 7 columns]


In [98]:
aave_OHLC_online.to_csv("data/final_data_aave.csv", index=False)

### 6. Fetch SOLANA

In [99]:
y_symbols = 'SOL-USD'

sol_OHLC_online = yf.Ticker(y_symbols).history('max')
sol_OHLC_online = sol_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
sol_OHLC_online.reset_index(inplace=True)
sol_OHLC_online['Date'] = pd.to_datetime(sol_OHLC_online['Date']).dt.date
sol_OHLC_online.sort_values(by='Date', inplace = True)
sol_OHLC_online = compute_tunh(sol_OHLC_online)
print(sol_OHLC_online)

           Date        Open        High         Low       Close      Volume  tunh
0    2020-04-10    0.832005    1.313487    0.694187    0.951054    87364276  96.0
1    2020-04-11    0.951054    1.049073    0.765020    0.776819    43862444  92.0
2    2020-04-12    0.785448    0.956670    0.762426    0.882507    38736897  79.0
3    2020-04-13    0.890760    0.891603    0.773976    0.777832    18211285  78.0
4    2020-04-14    0.777832    0.796472    0.628169    0.661925    16747614  16.0
...         ...         ...         ...         ...         ...         ...   ...
1821 2025-04-05  122.786430  122.786911  117.188805  120.258995  1667996329   NaN
1822 2025-04-06  120.258507  120.932678  104.004128  105.712013  3822458629   NaN
1823 2025-04-07  105.712013  111.471985   96.588066  106.902779  8644174246   1.0
1824 2025-04-08  106.905266  112.298935  103.144524  105.512192  5353641480   NaN
1825 2025-04-09  105.447334  107.803185  101.492363  104.298187  5831417344   NaN

[1826 rows x 7 

In [100]:
# Save to CSV
sol_OHLC_online.to_csv("data/final_data_sol.csv", index=False)

### 7. Fetch LTC

In [101]:
y_symbols = 'LTC-USD'

ltc_OHLC_online = yf.Ticker(y_symbols).history('max')
ltc_OHLC_online = ltc_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
ltc_OHLC_online.reset_index(inplace=True)
ltc_OHLC_online['Date'] = pd.to_datetime(ltc_OHLC_online['Date']).dt.date
ltc_OHLC_online.sort_values(by='Date', inplace = True)
ltc_OHLC_online = compute_tunh(ltc_OHLC_online)
print(ltc_OHLC_online)

           Date       Open       High        Low      Close      Volume   tunh
0    2014-09-17   5.085890   5.170770   4.965950   5.058550     3071840  292.0
1    2014-09-18   5.065430   5.065430   4.579960   4.685230     4569260  291.0
2    2014-09-19   4.687290   4.755820   4.254350   4.327770     3917450    4.0
3    2014-09-20   4.329200   4.616080   4.202190   4.286440     5490660    3.0
4    2014-09-21   4.263070   4.300130   4.154990   4.245920     2931220    1.0
...         ...        ...        ...        ...        ...         ...    ...
3853 2025-04-05  84.418518  84.877892  81.797729  82.711456   236343177    NaN
3854 2025-04-06  82.711197  82.997406  68.641136  70.428543   657828967    NaN
3855 2025-04-07  70.428658  72.354675  63.751862  70.965164  1102503251    1.0
3856 2025-04-08  70.970978  73.403198  68.422302  69.093231   520346488    NaN
3857 2025-04-09  69.085541  71.246185  66.788773  70.685234   632090176    NaN

[3858 rows x 7 columns]


In [102]:
ltc_OHLC_online.to_csv("data/final_data_ltc.csv", index=False)

### 8. Fetch XRP

In [103]:
y_symbols = 'XRP-USD'

xrp_OHLC_online = yf.Ticker(y_symbols).history('max')
xrp_OHLC_online = xrp_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
xrp_OHLC_online.reset_index(inplace=True)
xrp_OHLC_online['Date'] = pd.to_datetime(xrp_OHLC_online['Date']).dt.date
xrp_OHLC_online.sort_values(by='Date', inplace = True)
xrp_OHLC_online = compute_tunh(xrp_OHLC_online)
print(xrp_OHLC_online)

           Date      Open      High       Low     Close       Volume  tunh
0    2017-11-09  0.217911  0.221791  0.214866  0.217488    147916992   7.0
1    2017-11-10  0.218256  0.219068  0.205260  0.206483    141032992   6.0
2    2017-11-11  0.205948  0.214456  0.205459  0.210430    134503008   5.0
3    2017-11-12  0.210214  0.210214  0.195389  0.197339    251175008   2.0
4    2017-11-13  0.197472  0.204081  0.197456  0.203442    132567000   1.0
...         ...       ...       ...       ...       ...          ...   ...
2704 2025-04-05  2.128952  2.170936  2.114684  2.143731   2089759530   NaN
2705 2025-04-06  2.143723  2.156484  1.869135  1.917895   5158671865   NaN
2706 2025-04-07  1.917895  1.992880  1.640088  1.897754  14383598177   NaN
2707 2025-04-08  1.897754  1.966046  1.782009  1.794568   6468324898   NaN
2708 2025-04-09  1.793936  1.838947  1.727468  1.790493   7231097344   NaN

[2709 rows x 7 columns]


In [104]:
# Save to CSV
xrp_OHLC_online.to_csv("data/final_data_xrp.csv", index=False)

### 9. Fetch ADA

In [105]:
y_symbols = 'ADA-USD'

ada_OHLC_online = yf.Ticker(y_symbols).history('max')
ada_OHLC_online = ada_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
ada_OHLC_online.reset_index(inplace=True)
ada_OHLC_online['Date'] = pd.to_datetime(ada_OHLC_online['Date']).dt.date
ada_OHLC_online.sort_values(by='Date', inplace = True)
ada_OHLC_online = compute_tunh(ada_OHLC_online)
print(ada_OHLC_online)

           Date      Open      High       Low     Close      Volume  tunh
0    2017-11-09  0.025160  0.035060  0.025006  0.032053    18716200  17.0
1    2017-11-10  0.032219  0.033348  0.026451  0.027119     6766780  16.0
2    2017-11-11  0.026891  0.029659  0.025684  0.027437     5532220   9.0
3    2017-11-12  0.027480  0.027952  0.022591  0.023977     7280250   4.0
4    2017-11-13  0.024364  0.026300  0.023495  0.025808     4419440   1.0
...         ...       ...       ...       ...       ...         ...   ...
2704 2025-04-05  0.660436  0.666402  0.645755  0.654671   403628594   NaN
2705 2025-04-06  0.654671  0.654763  0.561787  0.572793   943427756   NaN
2706 2025-04-07  0.572803  0.596619  0.516226  0.585431  1985606963   1.0
2707 2025-04-08  0.585435  0.616657  0.554425  0.557888  1249643862   NaN
2708 2025-04-09  0.557809  0.577088  0.538733  0.559917  1348352256   NaN

[2709 rows x 7 columns]


In [106]:
# Save to CSV
ada_OHLC_online.to_csv("data/final_data_ada.csv", index=False)

### 10. Fetch DOGE

In [107]:
y_symbols = 'DOGE-USD'

doge_OHLC_online = yf.Ticker(y_symbols).history('max')
doge_OHLC_online = doge_OHLC_online.drop(columns=['Dividends', 'Stock Splits'], axis=1)
doge_OHLC_online.reset_index(inplace=True)
doge_OHLC_online['Date'] = pd.to_datetime(doge_OHLC_online['Date']).dt.date
doge_OHLC_online.sort_values(by='Date', inplace = True)
doge_OHLC_online = compute_tunh(doge_OHLC_online)
print(doge_OHLC_online)

           Date      Open      High       Low     Close      Volume  tunh
0    2017-11-09  0.001207  0.001415  0.001181  0.001415     6259550   1.0
1    2017-11-10  0.001421  0.001431  0.001125  0.001163     4246520  11.0
2    2017-11-11  0.001146  0.001257  0.001141  0.001201     2231080   4.0
3    2017-11-12  0.001189  0.001210  0.001002  0.001038     3288960   1.0
4    2017-11-13  0.001046  0.001212  0.001019  0.001211     2481270   1.0
...         ...       ...       ...       ...       ...         ...   ...
2704 2025-04-05  0.171098  0.171413  0.166378  0.169027   607481980   NaN
2705 2025-04-06  0.169027  0.169769  0.146452  0.148845  1267445532   NaN
2706 2025-04-07  0.148845  0.152600  0.131507  0.149521  3512035481   1.0
2707 2025-04-08  0.149520  0.156727  0.141514  0.142107  1829881699   NaN
2708 2025-04-09  0.142101  0.147728  0.137018  0.145728  1957285504   NaN

[2709 rows x 7 columns]


In [108]:
doge_OHLC_online.to_csv("data/final_data_doge.csv", index=False)